In [562]:
import numpy as np
import pandas as pd
from scipy.special import softmax

In [563]:
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META']
df_sims = {t: pd.read_csv(f"../price_sim/{t}_sim.csv") for t in tickers}

In [564]:
def sigmoid(x, k=1):
    return np.array(1 / (1 + np.exp(-k * x)))

In [565]:
def arctan(x, k=1):
    return np.array(np.arctan(k * x) / (np.pi / 2))

In [566]:
def auto_scale(x, decay=0.9):
    arr = x.copy()
    base_ratio = 1/arr.shape[1]
    for i in range(len(arr)):
        while True:
            atan_x = arctan(arr[i])
            arr[i] = (atan_x - atan_x.mean()) * (1-base_ratio) * decay + base_ratio
            if not (arr[i].max() > 1 or arr[i].min() < -1):
                break
    return arr

In [567]:
def sharpe_ratio(capital, risk_free=0.0):
    returns = np.diff(capital) / capital[:-1]
    excess = np.mean(returns - risk_free)
    std_dev = np.std(returns)
    return (excess / std_dev) * np.sqrt(252)

In [568]:
def multi_preprocess(df_sims, pool):
    length = len(df_sims[pool[0]])
    real_price = np.column_stack([df_sims[t]['real_price'] for t in pool] + [np.ones(length)])
    pred_delta = np.column_stack([df_sims[t]['pred_delta'] for t in pool] + [np.zeros(length)])
    pred_ratio = np.column_stack([df_sims[t]['pred_delta'] / df_sims[t]['real_price'].shift(1) for t in pool] + [np.zeros(length)])
    return pred_ratio, pred_delta, real_price

In [569]:
def trade(ratios, prices):
    capital = []
    currency = [1.0]
    position = [0.0]
    for ratio, price in zip(ratios[1:], prices[:-1]):
        money = currency[-1] + position[-1] * price
        assert money >= 0, "Bankruptcy"
        capital.append(money)
        nxt_cur = money * (1 - ratio)
        nxt_pos = money * ratio / price
        currency.append(nxt_cur)
        position.append(nxt_pos)
    money = currency[-1] + position[-1] * prices[-1]
    capital.append(money)

    return pd.DataFrame({
        'ratios': ratios,
        'prices': prices,
        'capital': capital,
        'currency': currency,
        'position': position
    })

In [570]:
def multi_trade(ratios, prices):
    capital = []
    position = [np.eye(prices.shape[1])[-1]]
    for ratio, price in zip(ratios[1:], prices[:-1]):
        if not capital:
            money = 1.0
        else:
            money = (position[-1] * price).sum()
        capital.append(money)
        assert money >= 0, "Bankruptcy"
        nxt_pos = money * ratio / price
        position.append(nxt_pos)
    money = (position[-1] * prices[-1]).sum()
    capital.append(money)
    
    return pd.DataFrame({
        'ratios': np.sum(ratios[:, :-1], axis=1),
        'prices': np.mean(prices[:, :-1], axis=1),
        'capital': capital,
        'position': position
    })

In [571]:
annual_risk_free = 0.04
daily_risk_free = (1+annual_risk_free) ** (1/252) - 1
daily_risk_free

0.0001556498627912628

In [ ]:
for key in df_sims:
    df_sim = df_sims[key]
    ratios1, prices1 = np.ones(len(df_sim), dtype=float), df_sim['real_price'].to_numpy()
    df_trade1 = trade(ratios1, prices1)
    ratios2, prices2 = sigmoid(df_sim['pred_delta'], 5), df_sim['real_price'].to_numpy()
    df_trade2 = trade(ratios2, prices2)
    ratios3, prices3 = arctan(df_sim['pred_delta'], 1), df_sim['real_price'].to_numpy()
    df_trade3 = trade(ratios3, prices3)
    print(f"Ticker: {key}")
    print(f"Always hold stock: (Sharpe){sharpe_ratio(df_trade1['capital'], daily_risk_free):.2f}, (Yield){df_trade1['capital'].iloc[-1]:.2f}")
    print(f"Single stock (no short selling): (Sharpe){sharpe_ratio(df_trade2['capital'], daily_risk_free):.2f}, (Yield){df_trade2['capital'].iloc[-1]:.2f}")
    print(f"Single stock (short selling): (Sharpe){sharpe_ratio(df_trade3['capital'], daily_risk_free):.2f}, (Yield){df_trade3['capital'].iloc[-1]:.2f}")

Ticket: AAPL
Always hold stock: (Sharpe)1.37, (Yield)1.75
Single stock (no short selling): (Sharpe)3.32, (Yield)2.69
Single stock (short selling): (Sharpe)3.86, (Yield)2.52
Ticket: GOOGL
Always hold stock: (Sharpe)1.21, (Yield)1.86
Single stock (no short selling): (Sharpe)4.27, (Yield)3.89
Single stock (short selling): (Sharpe)4.88, (Yield)3.90
Ticket: MSFT
Always hold stock: (Sharpe)1.43, (Yield)1.81
Single stock (no short selling): (Sharpe)3.35, (Yield)2.77
Single stock (short selling): (Sharpe)4.12, (Yield)3.24
Ticket: AMZN
Always hold stock: (Sharpe)1.31, (Yield)2.00
Single stock (no short selling): (Sharpe)3.74, (Yield)4.45
Single stock (short selling): (Sharpe)4.50, (Yield)4.51
Ticket: META
Always hold stock: (Sharpe)2.19, (Yield)4.34
Single stock (no short selling): (Sharpe)2.98, (Yield)5.81
Single stock (short selling): (Sharpe)3.22, (Yield)5.17


In [573]:
pred_ratio, pred_delta, real_price = multi_preprocess(df_sims, tickers)
ratios4, prices4 = softmax(pred_ratio * 5 * real_price.mean(), axis=1), real_price
df_trade4 = multi_trade(ratios4, prices4)

In [574]:
prices5 = real_price[:, :-1]
ratios5 = np.full(prices5.shape, 1/len(tickers))
df_trade5 = multi_trade(ratios5, prices5)

In [575]:
avg_delta, avg_price = np.mean(pred_delta[:, :-1], axis=1), np.mean(real_price[:, :-1], axis=1)
ratios6, prices6 = sigmoid(avg_delta, 5), avg_price
df_trade6 = trade(ratios6, prices6)

In [576]:
ratios7, prices7 = arctan(avg_delta, 1), avg_price
df_trade7 = trade(ratios7, prices7)

In [577]:
ratios8, prices8 = auto_scale(pred_ratio * 1 * real_price.mean()), real_price
df_trade8 = multi_trade(ratios8, prices8)

In [578]:
print(f"Portfolio")
print(f"Fixed Portfolio (always hold): (Sharpe){sharpe_ratio(df_trade5['capital'], daily_risk_free):.2f}, (Yield){df_trade5['capital'].iloc[-1]:.2f}")
print(f"Fixed Portfolio (no short selling): (Sharpe){sharpe_ratio(df_trade6['capital'], daily_risk_free):.2f}, (Yield){df_trade6['capital'].iloc[-1]:.2f}")
print(f"Dynanmic Portfolio (no short selling): (Sharpe){sharpe_ratio(df_trade4['capital'], daily_risk_free):.2f}, (Yield){df_trade4['capital'].iloc[-1]:.2f}")
print(f"Fixed Portfolio (short selling): (Sharpe){sharpe_ratio(df_trade7['capital'], daily_risk_free):.2f}, (Yield){df_trade7['capital'].iloc[-1]:.2f}")
print(f"Dynanmic Portfolio (short selling): (Sharpe){sharpe_ratio(df_trade8['capital'], daily_risk_free):.2f}, (Yield){df_trade8['capital'].iloc[-1]:.2f}")

Portfolio
Fixed Portfolio (always hold): (Sharpe)1.99, (Yield)2.27
Fixed Portfolio (no short selling): (Sharpe)4.12, (Yield)3.61
Dynanmic Portfolio (no short selling): (Sharpe)4.71, (Yield)11.14
Fixed Portfolio (short selling): (Sharpe)4.81, (Yield)3.17
Dynanmic Portfolio (short selling): (Sharpe)6.27, (Yield)23.62
